# 처음부터 ReAct 에이전트를 만드는 방법

!!! info "필수 조건"
    이 가이드는 다음에 대한 익숙함을 가정합니다:
    
    - [도구 호출 에이전트](../../concepts/agentic_concepts/#tool-calling-agent)
    - [채팅 모델](https://python.langchain.com/docs/concepts/chat_models/)
    - [메시지](https://python.langchain.com/docs/concepts/messages/)
    - [LangGraph 용어집](../../concepts/low_level/)

사전 구축된 ReAct 에이전트 [create_react_agent][langgraph.prebuilt.chat_agent_executor.create_react_agent]를 사용하는 것은 시작하기에 좋은 방법이지만, 때로는 더 많은 제어와 사용자 정의를 원할 수 있습니다. 이러한 경우 사용자 정의 ReAct 에이전트를 만들 수 있습니다. 이 가이드는 LangGraph를 사용하여 처음부터 ReAct 에이전트를 구현하는 방법을 보여줍니다.

## 설정

먼저 필요한 패키지를 설치하고 API 키를 설정하겠습니다:

In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

<div class="admonition tip">
     <p class="admonition-title">더 나은 디버깅을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
     <p style="padding-top: 5px;">
         LangSmith에 가입하여 문제를 빠르게 발견하고 LangGraph 프로젝트의 성능을 개선하세요. LangSmith를 사용하면 LangGraph로 구축한 LLM 앱을 디버그, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대해 자세히 알아보려면 <a href="https://docs.smith.langchain.com">문서</a>를 참조하세요. 
     </p>
 </div>

## ReAct 에이전트 생성

이제 필요한 패키지를 설치하고 환경 변수를 설정했으므로 ReAct 에이전트를 코딩할 수 있습니다!

### 그래프 상태 정의

이 예제에서는 가장 기본적인 ReAct 상태를 정의할 것이며, 이는 메시지 목록만 포함합니다.

특정 사용 사례에 대해서는 필요한 다른 상태 키를 자유롭게 추가하세요.

In [ ]:
from typing import (
    Annotated,
    Sequence,
    TypedDict,
)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    """에이전트의 상태입니다."""

    # add_messages는 reducer입니다
    # https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers 참조
    messages: Annotated[Sequence[BaseMessage], add_messages]

### 모델 및 도구 정의

다음으로 예제에 사용할 도구와 모델을 정의하겠습니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

model = ChatOpenAI(model="gpt-4o-mini")


@tool
def get_weather(location: str):
    """특정 위치의 날씨를 가져오기 위해 호출합니다."""
    # 이것은 실제 구현을 위한 플레이스홀더입니다
    # 하지만 LLM에게는 알리지 마세요 😊
    if any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
    else:
        return f"I am not sure what the weather is in {location}"


tools = [get_weather]

model = model.bind_tools(tools)

### 노드 및 엣지 정의

다음으로 노드와 엣지를 정의하겠습니다. 기본 ReAct 에이전트에는 모델 호출용 노드와 도구 사용용 노드 두 개만 있지만, 사용 사례에 더 잘 맞도록 이 기본 구조를 수정할 수 있습니다. 여기서 정의하는 도구 노드는 사전 구축된 [`ToolNode`](https://langchain-ai.github.io/langgraph/how-tos/tool-calling/)의 단순화된 버전으로, 몇 가지 추가 기능이 있습니다.

[구조화된 출력 추가](https://langchain-ai.github.io/langgraph/how-tos/react-agent-structured-output/)를 위한 노드 또는 일부 외부 작업(이메일 보내기, 캘린더 이벤트 추가 등)을 실행하기 위한 노드를 추가하고 싶을 수 있습니다. `call_model` 노드가 작동하는 방식과 `should_continue`가 도구를 호출할지 여부를 결정하는 방식을 변경하고 싶을 수도 있습니다 - 가능성은 무궁무진하며 LangGraph는 특정 사용 사례에 맞게 이 기본 구조를 쉽게 사용자 정의할 수 있게 해줍니다.

In [ ]:
import json
from langchain_core.messages import ToolMessage, SystemMessage
from langchain_core.runnables import RunnableConfig

tools_by_name = {tool.name: tool for tool in tools}


# 도구 노드 정의
def tool_node(state: AgentState):
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}


# 모델을 호출하는 노드 정의
def call_model(
    state: AgentState,
    config: RunnableConfig,
):
    # 이것은 'prompt' 매개변수로 create_react_agent를 사용자 정의하는 것과 유사하지만 더 유연합니다
    system_prompt = SystemMessage(
        "You are a helpful AI assistant, please respond to the users query to the best of your ability!"
    )
    response = model.invoke([system_prompt] + state["messages"], config)
    # 기존 목록에 추가될 것이므로 리스트를 반환합니다
    return {"messages": [response]}


# 계속할지 여부를 결정하는 조건부 엣지 정의
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    # 함수 호출이 없으면 종료합니다
    if not last_message.tool_calls:
        return "end"
    # 그렇지 않으면 계속합니다
    else:
        return "continue"

### 그래프 정의

이제 모든 노드와 엣지를 정의했으므로 그래프를 정의하고 컴파일할 수 있습니다. 더 많은 노드를 추가하거나 다른 엣지를 추가한 경우 특정 사용 사례에 맞게 이를 편집해야 합니다.

In [ ]:
from langgraph.graph import StateGraph, END

# 새 그래프 정의
workflow = StateGraph(AgentState)

# 순환할 두 개의 노드 정의
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# 엔트리포인트를 `agent`로 설정
# 이것은 이 노드가 첫 번째로 호출된다는 것을 의미합니다
workflow.set_entry_point("agent")

# 이제 조건부 엣지를 추가합니다
workflow.add_conditional_edges(
    # 먼저 시작 노드를 정의합니다. `agent`를 사용합니다.
    # 이것은 `agent` 노드가 호출된 후 취하는 엣지를 의미합니다.
    "agent",
    # 다음으로 어느 노드가 다음에 호출될지 결정하는 함수를 전달합니다.
    should_continue,
    # 마지막으로 매핑을 전달합니다.
    # 키는 문자열이고 값은 다른 노드입니다.
    # END는 그래프가 종료되어야 함을 나타내는 특수 노드입니다.
    # 일어나는 일은 `should_continue`를 호출하고 그 출력이
    # 이 매핑의 키와 일치됩니다.
    # 어떤 것과 일치하는지에 따라 해당 노드가 호출됩니다.
    {
        # `tools`이면 도구 노드를 호출합니다.
        "continue": "tools",
        # 그렇지 않으면 종료합니다.
        "end": END,
    },
)

# 이제 `tools`에서 `agent`로 일반 엣지를 추가합니다.
# 이것은 `tools`가 호출된 후 `agent` 노드가 다음에 호출된다는 것을 의미합니다.
workflow.add_edge("tools", "agent")

# 이제 그래프를 컴파일하고 시각화할 수 있습니다
graph = workflow.compile()

from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # 이것은 몇 가지 추가 종속성이 필요하며 선택 사항입니다
    pass

## ReAct 에이전트 사용

이제 React 에이전트를 만들었으므로 실제로 테스트해 봅시다!

In [ ]:
# 스트림을 깔끔하게 포맷하기 위한 헬퍼 함수
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


inputs = {"messages": [("user", "what is the weather in sf")]}
print_stream(graph.stream(inputs, stream_mode="values"))

완벽합니다! 그래프가 `get_weather` 도구를 올바르게 호출하고 도구에서 정보를 받은 후 사용자에게 응답합니다.